<a href="https://colab.research.google.com/github/SomdeepAcharyya/Recommender-Systems/blob/main/Singular_Value_Decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Singular Value Decomposition

In [ ]:
import pandas as pd
import scipy as sp
import cmake
import numpy as np
import warnings
from sklearn import svm
warnings.filterwarnings('ignore')
from sklearn.linear_model import orthogonal_mp_gram

In [ ]:
#from ksvd import ApproximateKSVD
from sklearn.decomposition import DictionaryLearning
import sklearn.utils._testing
from sklearn.utils._testing import assert_array_almost_equal
import numpy as np
import scipy as sp
from scipy.linalg import norm

In [ ]:
# Tripadvisor review Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/tripadvisor_reviews_with_country.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  tr = pd.read_csv(infile)
tr = tr.rename(columns={"username":"userId", "taObject":"itemId"})
pred_df = tr[['userId', 'itemId','rating', 'open', 'cons','extra','agree', 'neuro']]

In [ ]:
# amazon review dataset magazines csv
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Text_Digital_Music.json'
with open(path, encoding="utf-8", errors='ignore') as infile:
  az = pd.read_json(infile, lines=True, nrows=12000)
az = az.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})

In [ ]:
# amazon review dataset movies json
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_fashion_filtered.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  az = pd.read_csv(infile)
#am = am.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})

In [ ]:
# Tripadvisor review Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/pers_fashion_filtered.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  arr = pd.read_csv(infile)
arr = arr[['0','1','2','3','4']]
arr.columns=['open', 'cons','extra','agree', 'neuro']

In [ ]:
#tgt = az
src = az

In [ ]:
print(arr.shape, az.shape)
az = az[0:len(arr)]
print(arr.shape, az.shape)

(7841, 5) (7891, 6)
(7841, 5) (7841, 6)


In [ ]:
tgt_pers = arr[['open', 'cons', 'extra', 'agree', 'neuro']]
#tgt_pers[0]
pred_df = pd.DataFrame()
pred_df = tgt_pers

In [ ]:
pred_df['userId'] = src['userId']
pred_df['itemId'] = src['itemId']
pred_df['rating'] = src['rating']

In [ ]:
pred_df.open.value_counts()

# KSVD

In [ ]:
class ApproximateKSVD(object):
    def __init__(self, n_components, max_iter=100, tol=1e-4,
                 transform_n_nonzero_coefs=None):
        """
        Parameters
        ----------
        n_components:
            Number of dictionary elements
        max_iter:
            Maximum number of iterations
        tol:
            tolerance for error
        transform_n_nonzero_coefs:
            Number of nonzero coefficients to target
        """
        self.components_ = None
        self.max_iter = max_iter
        self.tol = tol
        self.n_components = n_components
        self.transform_n_nonzero_coefs = transform_n_nonzero_coefs

    def _update_dict(self, X, D, gamma):
        for j in range(self.n_components):
            I = gamma[:, j] > 0
            if np.sum(I) == 0:
                continue

            D[j, :] = 0
            g = gamma[I, j].T
            r = X[I, :] - gamma[I, :].dot(D)
            d = r.T.dot(g)
            d /= np.linalg.norm(d)
            g = r.dot(d)
            D[j, :] = d
            gamma[I, j] = g.T
        return D, gamma

    def _initialize(self, X):
        if min(X.shape) < self.n_components:
            D = np.random.randn(self.n_components, X.shape[1])
        else:
            u, s, vt = sp.sparse.linalg.svds(X, k=self.n_components)
            D = np.dot(np.diag(s), vt)
        D /= np.linalg.norm(D, axis=1)[:, np.newaxis]
        return D

    def _transform(self, D, X):
        gram = D.dot(D.T)
        Xy = D.dot(X.T)

        n_nonzero_coefs = self.transform_n_nonzero_coefs
        if n_nonzero_coefs is None:
            n_nonzero_coefs = int(0.1 * X.shape[1])

        return orthogonal_mp_gram(
            gram, Xy, n_nonzero_coefs=n_nonzero_coefs).T

    def fit(self, X):
        """
        Parameters
        ----------
        X: shape = [n_samples, n_features]
        """
        D = self._initialize(X)
        for i in range(self.max_iter):
            gamma = self._transform(D, X)
            e = np.linalg.norm(X - gamma.dot(D))
            if e < self.tol:
                break
            D, gamma = self._update_dict(X, D, gamma)

        self.components_ = D
        return self

    def transform(self, X):
        return self._transform(self.components_, X)


In [ ]:
def test_initialize_with_small_n_features(X, k, Y):
    N = X.shape[0]
    n_components = k 
    n_features = X.shape[1]
    #X = np.random.randn(N, n_features)
    dico = ApproximateKSVD(n_components=n_components, transform_n_nonzero_coefs=n_components)
    dico.fit(X)
    dictionary = dico.fit(X).components_
    gamma = dico.transform(X)
    pred_d = dico.fit(Y).components_
    pred_g = dico.transform(Y)
    pred = pred_g @ pred_d
    return dictionary, gamma, pred

In [ ]:
def test_fit():
    np.random.seed(0)
    N = 1000
    L = 64
    n_features = 128
    B = np.array(sp.sparse.random(N, L, density=0.5).todense())
    D = np.random.randn(L, n_features)
    X = np.dot(B, D)
    dico = ApproximateKSVD(n_components=L, transform_n_nonzero_coefs=L)
    dico.fit(X)
    gamma = dico.transform(X)
    assert_array_almost_equal(X, gamma.dot(dico.components_))

In [ ]:
def test_size():
    np.random.seed(0)
    N = 50
    L = 12
    n_features = 16
    D = np.random.randn(L, n_features)
    B = np.array(sp.sparse.random(N, L, density=0.5).todense())
    X = np.dot(B, D)
    dico1 = ApproximateKSVD(n_components=L, transform_n_nonzero_coefs=L)
    dico1.fit(X)
    gamma1 = dico1.transform(X)
    e1 = norm(X - gamma1.dot(dico1.components_))

    dico2 = DictionaryLearning(n_components=L, transform_n_nonzero_coefs=L)
    dico2.fit(X)
    gamma2 = dico2.transform(X)
    e2 = norm(X - gamma2.dot(dico2.components_))

    assert dico1.components_.shape == dico2.components_.shape
    assert gamma1.shape == gamma2.shape
    assert e1 < e2

In [ ]:
# rating matrix wrt user u
ru = src.pivot_table(index='userId',columns='itemId',values='rating')
ru = ru.fillna(0)
ru_m = ru > 0
ru_m = ru_m.replace(True, 1)
ru_m = ru_m.replace(False, 0)
ru = np.array(ru)
ru_m = np.array(ru_m)

In [ ]:
train_size = 0.9
df_copy = src.copy()
train_set = df_copy.sample(frac=train_size).reset_index()
user_features_train = np.array(train_set[['open', 'cons', 'extra', 'agree', 'neuro']].fillna(0))
test_set = df_copy.drop(train_set.index).reset_index()
user_features_test = np.array(test_set[['open', 'cons', 'extra', 'agree', 'neuro']].fillna(0))
user_features_unique = np.array(src.groupby(by='userId').mean()[['open', 'cons', 'extra', 'agree', 'neuro']].fillna(0))
n_dims = 10
n_features = 10
parameters = {}

In [ ]:
    user_to_row = {}
    item_to_column = {}
    uniq_users = np.unique(train_set.userId)
    uniq_items = np.unique(train_set.itemId)

    for i, user_id in enumerate(uniq_users):
        user_to_row[user_id] = i

    for j, item_id in enumerate(uniq_items):
        item_to_column[item_id] = j

    n_users = len(uniq_users)
    n_items = len(uniq_items)
    rating =[]
    for i in range(n_users):
      rating.append(np.average([i]))

    R = np.zeros((n_users, n_items))
    for index, row in train_set.iterrows():
        i = user_to_row[row.userId]
        j = item_to_column[row.itemId]
        R[i, j] = row.rating

In [ ]:
    user_to_row = {}
    item_to_column = {}
    uniq_users = np.unique(test_set.userId)
    uniq_items = np.unique(test_set.itemId)

    for i, user_id in enumerate(uniq_users):
        user_to_row[user_id] = i

    for j, item_id in enumerate(uniq_items):
        item_to_column[item_id] = j

    n_users = len(uniq_users)
    n_items = len(uniq_items)
    rating =[]
    for i in range(n_users):
      rating.append(np.average([i]))

    Rt = np.zeros((n_users, n_items))
    for index, row in test_set.iterrows():
        i = user_to_row[row.userId]
        j = item_to_column[row.itemId]
        Rt[i, j] = row.rating

In [ ]:
d, g, p = test_initialize_with_small_n_features(R, 10, Rt)

In [ ]:
pred = g@d

In [ ]:
p

array([[ 6.46532366e-06, -4.28261641e-18,  1.95843342e-04, ...,
         1.09138150e-19, -1.03862772e-05, -6.91243322e-17],
       [-6.56135567e-09, -4.33398786e-18, -2.13588181e-07, ...,
         1.10447299e-19,  7.11075204e-04, -6.99535023e-17],
       [ 1.14987969e-07,  9.72937553e-20,  3.64012027e-06, ...,
        -2.47943300e-21,  2.41260042e-07,  1.57038717e-18],
       ...,
       [-6.11345124e-12, -5.43425251e-20, -1.85610347e-10, ...,
         1.38486432e-21,  1.58246035e-05, -8.77125196e-19],
       [-6.20530675e-11,  3.92845451e-22, -1.98667064e-09, ...,
        -1.00112692e-23,  8.04738036e-09,  6.34079191e-21],
       [ 5.51852909e-09, -4.03021557e-18,  1.57454620e-07, ...,
         1.02705970e-19, -8.59118240e-06, -6.50504115e-17]])

In [ ]:
pred = pd.DataFrame(pred)
p_n = pred > 2.5
p_n = p_n.replace(True, 1)
p_n = p_n.replace(False, 0)
p_n = np.array(p_n)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
mae = mean_absolute_error(Rt, p)
rmse = np.sqrt(mean_squared_error(Rt, p))
print(mae, rmse)
# 0.004541958435804962
# 0.0010510987831314456

0.03246389874443447 0.3519122274304687


# SVD++

In [ ]:
import time
from functools import wraps
from math import trunc
import numpy as np
import pandas as pd
from numba import njit
#from .__version__ import __version__
__version__ = '0.0.1.dev1'
__all__ = ['get_version', '_timer','SVD','_compute_val_metrics', '_initialization','_run_epoch','_shuffle']

In [ ]:
def get_version():
    return __version__

    
def _timer(text=''):
    """Decorator, prints execution time of the decorated function.
    Parameters
    ----------
    text : str
        Text to print before time display.
    Examples
    --------
    >>> @_timer(text='Greetings took ')
    ... def say_hi():
    ...    time.sleep(1)
    ...    print("Hey! What's up!")
    ...
    >>> say_hi()
    Hey! What's up!
    Greetings took 1 sec
    """
    def decorator(func):

        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time.time()
            result = func(*args, **kwargs)
            end = time.time()

            hours = trunc((end - start) / 3600)
            minutes = trunc((end - start) / 60)
            seconds = round((end - start) % 60)

            if hours > 1:
                print(text + '{} hours {} min and {} sec'.format(hours,
                                                                 minutes,
                                                                 seconds))
            elif hours == 1:
                print(text + '{} hour {} min and {} sec'.format(hours, minutes,
                                                                seconds))
            elif minutes >= 1:
                print(text + '{} min and {} sec'.format(minutes, seconds))
            else:
                print(text + '{} sec'.format(seconds))

            return result

        return wrapper

    return decorator

In [ ]:
@njit
def _shuffle(X):
    np.random.shuffle(X)
    return X


@njit
def _initialization(n_users, n_items, n_factors):
    """Initializes biases and latent factor matrices.
    Parameters
    ----------
    n_users : int
        Number of unique users.
    n_items : int
        Number of unique items.
    n_factors : int
        Number of factors.
    Returns
    -------
    bu : numpy.array
        User biases vector.
    bi : numpy.array
        Item biases vector.
    pu : numpy.array
        User latent factors matrix.
    qi : numpy.array
        Item latent factors matrix.
    """
    bu = np.zeros(n_users)
    bi = np.zeros(n_items)

    pu = np.random.normal(0, .1, (n_users, n_factors))
    qi = np.random.normal(0, .1, (n_items, n_factors))

    return bu, bi, pu, qi


@njit
def _run_epoch(X, bu, bi, pu, qi, global_mean, n_factors, lr, reg):
    """Runs an epoch, updating model weights (pu, qi, bu, bi).
    Parameters
    ----------
    X : numpy.array
        Training set.
    bu : numpy.array
        User biases vector.
    bi : numpy.array
        Item biases vector.
    pu : numpy.array
        User latent factors matrix.
    qi : numpy.array
        Item latent factors matrix.
    global_mean : float
        Ratings arithmetic mean.
    n_factors : int
        Number of latent factors.
    lr : float
        Learning rate.
    reg : float
        L2 regularization factor.
    Returns:
    --------
    bu : numpy.array
        User biases vector.
    bi : numpy.array
        Item biases vector.
    pu : numpy.array
        User latent factors matrix.
    qi : numpy.array
        Item latent factors matrix.
    """
    for i in range(X.shape[0]):
        user, item, rating = int(X[i, 0]), int(X[i, 1]), X[i, 2]

        # Predict current rating
        pred = global_mean + bu[user] + bi[item]

        for factor in range(n_factors):
            pred += pu[user, factor] * qi[item, factor]

        err = rating - pred

        # Update biases
        bu[user] += lr * (err - reg * bu[user])
        bi[item] += lr * (err - reg * bi[item])

        # Update latent factors
        for factor in range(n_factors):
            puf = pu[user, factor]
            qif = qi[item, factor]

            pu[user, factor] += lr * (err * qif - reg * puf)
            qi[item, factor] += lr * (err * puf - reg * qif)

    return bu, bi, pu, qi


@njit
def _compute_val_metrics(X_val, bu, bi, pu, qi, global_mean, n_factors):
    """Computes validation metrics (loss, rmse, and mae).
    Parameters
    ----------
    X_val : numpy.array
        Validation set.
    bu : numpy.array
        User biases vector.
    bi : numpy.array
        Item biases vector.
    pu : numpy.array
        User latent factors matrix.
    qi : numpy.array
        Item latent factors matrix.
    global_mean : float
        Ratings arithmetic mean.
    n_factors : int
        Number of latent factors.
    Returns
    -------
    loss, rmse, mae : tuple of floats
        Validation loss, rmse and mae.
    """
    residuals = []

    for i in range(X_val.shape[0]):
        user, item, rating = int(X_val[i, 0]), int(X_val[i, 1]), X_val[i, 2]
        pred = global_mean

        if user > -1:
            pred += bu[user]

        if item > -1:
            pred += bi[item]

        if (user > -1) and (item > -1):
            for factor in range(n_factors):
                pred += pu[user, factor] * qi[item, factor]

        residuals.append(rating - pred)

    residuals = np.array(residuals)
    loss = np.square(residuals).mean()
    rmse = np.sqrt(loss)
    mae = np.absolute(residuals).mean()
    print("rmse", rmse, "mae", mae)

    return loss, rmse, mae

In [ ]:
class SVD:
    """Implements Simon Funk famous SVD algorithm.
    Parameters
    ----------
    lr : float, default=.005
        Learning rate.
    reg : float, default=.02
        L2 regularization factor.
    n_epochs : int, default=20
        Number of SGD iterations.
    n_factors : int, default=100
        Number of latent factors.
    early_stopping : bool, default=False
        Whether or not to stop training based on a validation monitoring.
    shuffle : bool, default=False
        Whether or not to shuffle the training set before each epoch.
    min_delta : float, default=.001
        Minimun delta to argue for an improvement.
    min_rating : int, default=1
        Minimum value a rating should be clipped to at inference time.
    max_rating : int, default=5
        Maximum value a rating should be clipped to at inference time.
    Attributes
    ----------
    user_mapping_ : dict
        Maps user ids to their indexes.
    item_mapping_ : dict
        Maps item ids to their indexes.
    global_mean_ : float
        Ratings arithmetic mean.
    pu_ : numpy.array
        User latent factors matrix.
    qi_ : numpy.array
        Item latent factors matrix.
    bu_ : numpy.array
        User biases vector.
    bi_ : numpy.array
        Item biases vector.
    metrics_ : pandas.DataFrame
        Validation metrics at each epoch. Column names are 'Loss', 'RMSE', and
        'MAE'.
    """

    def __init__(self, lr=.001, reg=.01, n_epochs=100, n_factors=40,
                 early_stopping=False, shuffle=False, min_delta=.001,
                 min_rating=1, max_rating=5):

        self.lr = lr
        self.reg = reg
        self.n_epochs = n_epochs
        self.n_factors = n_factors
        self.early_stopping = early_stopping
        self.shuffle = shuffle
        self.min_delta = min_delta
        self.min_rating = min_rating
        self.max_rating = max_rating

    @_timer(text='\nTraining took ')
    def fit(self, X, X_val=None):
        """Learns model weights from input data.
        Parameters
        ----------
        X : pandas.DataFrame
            Training set, must have 'u_id' for user ids, 'i_id' for item ids,
            and 'rating' column names.
        X_val : pandas.DataFrame, default=None
            Validation set with the same column structure as X.
        Returns
        -------
        self : SVD object
            The current fitted object.
        """
        X = self._preprocess_data(X)

        if X_val is not None:
            X_val = self._preprocess_data(X_val, train=False, verbose=False)
            self._init_metrics()

        self.global_mean_ = np.mean(X[:, 2])
        self._run_sgd(X, X_val)

        return self

    def _preprocess_data(self, X, train=True, verbose=True):
        """Maps user and item ids to their indexes.
        Parameters
        ----------
        X : pandas.DataFrame
            Dataset, must have 'u_id' for user ids, 'i_id' for item ids, and
            'rating' column names.
        train : boolean
            Whether or not X is the training set or the validation set.
        Returns
        -------
        X : numpy.array
            Mapped dataset.
        """
        print('Preprocessing data...\n')
        X = X.copy()

        if train:  # Mappings have to be created
            user_ids = X['u_id'].unique().tolist()
            item_ids = X['i_id'].unique().tolist()

            n_users = len(user_ids)
            n_items = len(item_ids)

            user_idx = range(n_users)
            item_idx = range(n_items)

            self.user_mapping_ = dict(zip(user_ids, user_idx))
            self.item_mapping_ = dict(zip(item_ids, item_idx))

        X['u_id'] = X['u_id'].map(self.user_mapping_)
        X['i_id'] = X['i_id'].map(self.item_mapping_)

        # Tag validation set unknown users/items with -1 (enables
        # `fast_methods._compute_val_metrics` detecting them)
        X.fillna(-1, inplace=True)

        X['u_id'] = X['u_id'].astype(np.int32)
        X['i_id'] = X['i_id'].astype(np.int32)

        return X[['u_id', 'i_id', 'rating']].values

    def _init_metrics(self):
        metrics = np.zeros((self.n_epochs, 3), dtype=float)
        self.metrics_ = pd.DataFrame(metrics, columns=['Loss', 'RMSE', 'MAE'])

    def _run_sgd(self, X, X_val):
        """Runs SGD algorithm, learning model weights.
        Parameters
        ----------
        X : numpy.array
            Training set, first column must be user indexes, second one item
            indexes, and third one ratings.
        X_val : numpy.array or None
            Validation set with the same structure as X.
        """
        n_users = len(np.unique(X[:, 0]))
        n_items = len(np.unique(X[:, 1]))

        bu, bi, pu, qi = _initialization(n_users, n_items, self.n_factors)

        # Run SGD
        for epoch_ix in range(self.n_epochs):
            start = self._on_epoch_begin(epoch_ix)

            if self.shuffle:
                X = _shuffle(X)

            bu, bi, pu, qi = _run_epoch(X, bu, bi, pu, qi, self.global_mean_,
                                        self.n_factors, self.lr, self.reg)

            if X_val is not None:
                self.metrics_.loc[epoch_ix, :] = _compute_val_metrics(
                                                     X_val, bu, bi, pu, qi,
                                                     self.global_mean_,
                                                     self.n_factors
                                                 )
                self._on_epoch_end(start,
                                   self.metrics_.loc[epoch_ix, 'Loss'],
                                   self.metrics_.loc[epoch_ix, 'RMSE'],
                                   self.metrics_.loc[epoch_ix, 'MAE'])

                if self.early_stopping:
                    val_rmse = self.metrics_['RMSE'].tolist()
                    if self._early_stopping(val_rmse, epoch_ix,
                                            self.min_delta):
                        break

            else:
                self._on_epoch_end(start)

        self.bu_ = bu
        self.bi_ = bi
        self.pu_ = pu
        self.qi_ = qi
        #print(self.metrics_['RMSE'], self.metrics_['MAE'])

    def predict(self, X, clip=True):
        """Returns estimated ratings of several given user/item pairs.
        Parameters
        ----------
        X : pandas.DataFrame
            Storing all user/item pairs we want to predict the ratings. Must
            contains columns labeled 'u_id' and 'i_id'.
        clip : bool, default=True
            Whether to clip the predictions or not.
        Returns
        -------
        predictions : list
            Predictions belonging to the input user/item pairs.
        """
        return [
            self.predict_pair(u_id, i_id, clip)
            for u_id, i_id in zip(X['u_id'], X['i_id'])
        ]

    def predict_pair(self, u_id, i_id, clip=True):
        """Returns the model rating prediction for a given user/item pair.
        Parameters
        ----------
        u_id : int
            A user id.
        i_id : int
            An item id.
        clip : bool, default=True
            Whether to clip the prediction or not.
        Returns
        -------
        pred : float
            The estimated rating for the given user/item pair.
        """
        user_known, item_known = False, False
        pred = self.global_mean_

        if u_id in self.user_mapping_:
            user_known = True
            u_ix = self.user_mapping_[u_id]
            pred += self.bu_[u_ix]

        if i_id in self.item_mapping_:
            item_known = True
            i_ix = self.item_mapping_[i_id]
            pred += self.bi_[i_ix]

        if user_known and item_known:
            pred += np.dot(self.pu_[u_ix], self.qi_[i_ix])

        if clip:
            pred = self.max_rating if pred > self.max_rating else pred
            pred = self.min_rating if pred < self.min_rating else pred

        return pred

    def _early_stopping(self, val_rmse, epoch_idx, min_delta):
        """Returns True if validation rmse is not improving.
        Last rmse (plus `min_delta`) is compared with the second to last.
        Parameters
        ----------
        val_rmse : list
            Validation RMSEs.
        min_delta : float
            Minimun delta to argue for an improvement.
        Returns
        -------
        early_stopping : bool
            Whether to stop training or not.
        """
        if epoch_idx > 0:
            if val_rmse[epoch_idx] + min_delta > val_rmse[epoch_idx-1]:
                self.metrics_ = self.metrics_.loc[:(epoch_idx+1), :]
                return True
        return False

    def _on_epoch_begin(self, epoch_ix):
        """Displays epoch starting log and returns its starting time.
        Parameters
        ----------
        epoch_ix : int
            Epoch index.
        Returns
        -------
        start : float
            Starting time of the current epoch.
        """
        start = time.time()
        end = '  | ' if epoch_ix < 9 else ' | '
        #print('Epoch {}/{}'.format(epoch_ix + 1, self.n_epochs), end=end)

        return start

    def _on_epoch_end(self, start, val_loss=None, val_rmse=None, val_mae=None):
        """Displays epoch ending log.
        If self.verbose, computes and displays validation metrics (loss, rmse,
        and mae).
        Parameters
        ----------
        start : float
            Starting time of the current epoch.
        val_loss : float, default=None
            Validation loss.
        val_rmse : float, default=None
            Validation rmse.
        val_mae : float, default=None
            Validation mae.
        """
        end = time.time()

        if val_loss is not None:
          pass;
            #print(f'val_loss: {val_loss:.2f}', end=' - ')
            #print(f'val_rmse: {val_rmse:.2f}', end=' - ')
            #print(f'val_mae: {val_mae:.2f}', end=' - ')

        #print(f'took {end - start:.1f} sec')


In [ ]:
# rating matrix wrt user u
ru = pred_df.pivot_table(index='userId',columns='itemId',values='rating')
ru = ru.fillna(0)
ru_m = ru > 0
ru_m = ru_m.replace(True, 1)
ru_m = ru_m.replace(False, 0)
ru = np.array(ru)
ru_m = np.array(ru_m)

In [ ]:
train_size = 0.9
df_copy = pred_df.copy()
train_set = df_copy.sample(frac=train_size).reset_index()
user_features_train = np.array(train_set[['open', 'cons', 'extra', 'agree', 'neuro']].fillna(0))
test_set = df_copy.drop(train_set.index).reset_index()
user_features_test = np.array(test_set[['open', 'cons', 'extra', 'agree', 'neuro']].fillna(0))
user_features_unique = np.array(pred_df.groupby(by='userId').mean()[['open', 'cons', 'extra', 'agree', 'neuro']].fillna(0))
n_dims = 30
n_features = 5
parameters = {}

In [ ]:
    user_to_row = {}
    item_to_column = {}
    uniq_users = np.unique(train_set.userId)
    uniq_items = np.unique(train_set.itemId)

    for i, user_id in enumerate(uniq_users):
        user_to_row[user_id] = i

    for j, item_id in enumerate(uniq_items):
        item_to_column[item_id] = j

    n_users = len(uniq_users)
    n_items = len(uniq_items)
    rating =[]
    for i in range(n_users):
      rating.append(np.average([i]))

    R = np.zeros((n_users, n_items))
    for index, row in train_set.iterrows():
        i = user_to_row[row.userId]
        j = item_to_column[row.itemId]
        R[i, j] = row.rating

In [ ]:
    user_to_row = {}
    item_to_column = {}
    uniq_users = np.unique(test_set.userId)
    uniq_items = np.unique(test_set.itemId)

    for i, user_id in enumerate(uniq_users):
        user_to_row[user_id] = i

    for j, item_id in enumerate(uniq_items):
        item_to_column[item_id] = j

    n_users = len(uniq_users)
    n_items = len(uniq_items)
    rating =[]
    for i in range(n_users):
      rating.append(np.average([i]))

    Rt = np.zeros((n_users, n_items))
    for index, row in test_set.iterrows():
        i = user_to_row[row.userId]
        j = item_to_column[row.itemId]
        Rt[i, j] = row.rating

In [ ]:
df = train_set[['userId', 'itemId', 'rating']]
df.columns=['u_id', 'i_id', 'rating']

In [ ]:
df_val = test_set[['userId', 'itemId', 'rating']]
df_val.columns=['u_id', 'i_id', 'rating']

In [ ]:
svd = SVD(lr=0.001, reg=0.0001, n_epochs=100, n_factors=10, shuffle=False, min_rating=1, max_rating=5)

In [ ]:
svd.fit(X=df, X_val=df_val)

Preprocessing data...

Preprocessing data...

rmse 1.0567729467768796 mae 0.8053163035350284
rmse 1.051419451423391 mae 0.8017110358016192
rmse 1.0463627358398246 mae 0.7985575981198763
rmse 1.041581285761975 mae 0.7957585788322664
rmse 1.0370551121343774 mae 0.793217621941852
rmse 1.0327656550479243 mae 0.7908646303133726
rmse 1.0286956917995937 mae 0.7886540522906789
rmse 1.0248292491877546 mae 0.7865712002813467
rmse 1.0211515201033232 mae 0.7845920244318076
rmse 1.0176487844297917 mae 0.7826840546979221
rmse 1.014308334224258 mae 0.7808475010245334
rmse 1.0111184031167586 mae 0.7790678858976008
rmse 1.0080680998364584 mae 0.7773408132863006
rmse 1.005147345749835 mae 0.7756569087973526
rmse 1.0023468162775948 mae 0.7740104208221024
rmse 0.999657886043124 mae 0.772401406265492
rmse 0.9970725775950704 mae 0.7708265188970119
rmse 0.99458351354006 mae 0.7692899651601133
rmse 0.9921838719175322 mae 0.767789932705952
rmse 0.989867344647321 mae 0.7663176700798441
rmse 0.9876280988812377 m

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
pred = svd.predict(df_val)
mae = mean_absolute_error(df_val['rating'], pred)
rmse = mean_squared_error(df_val['rating'], pred)
print(mae, rmse)

0.6849643586112794 0.7812732900551826


In [ ]:
svd.metrics_['MAE']

In [ ]:
#tuning

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
lr = [0.01, 0.001, 0.0001]
K = [10, 20, 30, 40]
lambda_p = [0.001, 0.003, 0.005, 0.007]

In [ ]:
for l in lr:
  for k in K:
    for reg in lambda_p:
      svd = SVD(lr=l, reg=reg, n_epochs=150, n_factors=k, shuffle=False, min_rating=1, max_rating=5)
      svd.fit(X=df, X_val=df_val)
      pred = svd.predict(df_val)
      mae = mean_absolute_error(df_val['rating'], pred)
      rmse = mean_squared_error(df_val['rating'], pred)
      print(l, reg, k, mae, rmse)

Preprocessing data...

Preprocessing data...


Training took 0 sec
0.01 0.001 10 0.09719960166416113 0.13261182625774198
Preprocessing data...

Preprocessing data...


Training took 0 sec
0.01 0.003 10 0.10767117759650323 0.13988939769515854
Preprocessing data...

Preprocessing data...


Training took 0 sec
0.01 0.005 10 0.10967723182158011 0.1303204301494839
Preprocessing data...

Preprocessing data...


Training took 0 sec
0.01 0.007 10 0.11064583948633475 0.1301593477415536
Preprocessing data...

Preprocessing data...


Training took 0 sec
0.01 0.001 20 0.09798938947423688 0.13184486064261067
Preprocessing data...

Preprocessing data...


Training took 0 sec
0.01 0.003 20 0.10363431410303994 0.12993024781231652
Preprocessing data...

Preprocessing data...


Training took 0 sec
0.01 0.005 20 0.10734481969939637 0.12798941910254413
Preprocessing data...

Preprocessing data...


Training took 0 sec
0.01 0.007 20 0.11155931763618608 0.12992452029650242
Preprocessing data...

Preprocessi

# SVD

In [ ]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.5 MB 472 kB/s 


In [ ]:
# rating matrix wrt user u
ru_src = src.pivot_table(index='userId',columns='itemId',values='rating')    # U
ru_src = ru_src.fillna(0)
ru_m_src = ru_src > 0
ru_m_src = ru_m_src.replace(True, 1)
ru_m_src = ru_m_src.replace(False, 0)
ru_src = np.array(ru_src)
ru_m_src = np.array(ru_m_src)   # U

In [ ]:
# matrix factorization(MF) via singular value decomposition(SVD) ad bm25_weight

import scipy.sparse.linalg
from implicit.nearest_neighbours import bm25_weight

k = 10

us, x2, vs = scipy.sparse.linalg.svds(bm25_weight(ru_src), k)

In [ ]:
us.shape

(1098, 10)

In [ ]:
train_size = 0.9
df_copy = tr.copy()
train_set = df_copy.sample(frac=train_size).reset_index()
user_features_train = np.array(train_set[['open', 'cons', 'extra', 'agree', 'neuro']].reset_index().fillna(0))
test_set = df_copy.drop(train_set.index).reset_index()
user_features_test = np.array(test_set[['open', 'cons', 'extra', 'agree', 'neuro']].reset_index().fillna(0))

In [ ]:
# rating matrix wrt user u
ru_src = test_set.pivot_table(index='userId',columns='itemId',values='rating')    # U
ru_src = ru_src.fillna(0)
ru_m_src = ru_src > 0
ru_m_src = ru_m_src.replace(True, 1)
ru_m_src = ru_m_src.replace(False, 0)
ru_src = np.array(ru_src)
ru_m_src = np.array(ru_m_src)   # U

In [ ]:
arr = test_set.pivot_table(index='userId',columns='itemId',values='rating').fillna(0).reset_index()
cols = arr.columns
cols = cols[1:len(arr.T)]

In [ ]:
rows = arr.userId

In [ ]:
src_group = src.groupby(by='userId').mean().reset_index().reset_index()

In [ ]:
ids = []
for i in arr.userId:
  for j in range(len(src_group)):
    if i == src_group.userId[j]:
      ids.append(src_group['index'][j])

In [ ]:
pred = []
for i in range(len(rows)):
  row = []
  for j in range(len(cols)):
    x = np.array(vs.T[0]).reshape(k,1)
    y = np.array(us[ids[0]]).reshape(1,k)
    p = y @ x
    row.append(p)
  pred.append(row)

In [ ]:
pred = np.array(pred).reshape(len(pred), len(pred[0]))

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
mae = mean_absolute_error(ru_src, pred)
rmse = mean_squared_error(ru_src, pred)
print(mae, rmse)

0.03455038104534231 0.14983418538841162


In [ ]:
ru_src

array([[0., 0., 0., ..., 0., 0., 3.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])